# Workflow to test density matrix merge
The function cry_combine_density is usually part of crystal_functions.file_readwrite. However, for testing purposes it's copied into the repository file density.py.

In [16]:
from density import cry_combine_density #This is the test version
from crystal_functions.execute import runcry, runprop
from crystal_functions.file_readwrite import Crystal_output, write_cry_input, write_cry_properties, Crystal_input
from crystal_functions.calculate import cry_ads_energy
from crystal_functions.adsorb import sub_ads_indices

# pymatgen imports
from pymatgen.core.structure import Molecule, Structure, Lattice
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor

from ase.visualize import view

import pandas as pd
import shutil 
import numpy as np

### Find a suitable molecular database

In [17]:
adsorbate = Molecule('HHO',[[0.76,0.00,0.50],[-0.76, 0.00,0.50],[0.0, 0.0, 0.0]])
#adsorbate = Molecule('O',[[0.0, 0.0, 0.0]])
with MPRester("My_ID") as m:    
    rutile = m.get_structure_by_material_id("mp-2657")
    mgo = m.get_structure_by_material_id("mp-1265")
    anatase = m.get_structure_by_material_id("mp-390")
bulk = mgo

### Make these into loops over substrates and adsorbates

In [18]:
bulk = SpacegroupAnalyzer(bulk).get_conventional_standard_structure()

In [19]:
substrate = (SlabGenerator(bulk, (1,0,0), 10., 10., center_slab=True, max_normal_search=5).get_slab())

For now we adsorb on one surface only because I'm still working on the adsorb_both_surfaces

In [23]:
system = AdsorbateSiteFinder(substrate).generate_adsorption_structures(adsorbate,repeat=[1,1,1])

In [5]:
#system = AdsorbateSiteFinder(substrate).adsorb_both_surfaces(adsorbate,repeat=[1,1,1])[0]

In [6]:
geom_block = ['Adsorption tests\n','EXTERNAL\n','EXTPRT\n','OPTGEOM\n','END\n']
bs_block = ['BASISSET\n', 'POB-DZVP\n']
func_block = ['DFT\n', 'B3LYP\n', 'XXLGRID\n', 'ENDDFT\n']
scf_block = [['TOLINTEG\n', '7 7 7 7 14\n'],
             ['SHRINK\n', '12 24\n'],
             ['MAXCYCLE\n', '200\n'],
             ['FMIXING\n', '70\n'],
             'DIIS\n',
             'ENDSCF\n']

### Change the path

In [7]:
path = 'data/test'
input_name = path+str(substrate.composition).replace(" ", "")+'_'+str(adsorbate.composition).replace(" ", "")+'_' \
               +''.join(str(x) for x in substrate.miller_index)+'.d12'
file_names = input_name[:-4]
write_cry_input(input_name,crystal_blocks=[geom_block,bs_block,func_block,scf_block],external_obj=system)

# Run on cx1 

### Check this is fine on cx1
We might need to change the names if we are testing different materials because they can't all be called adsrobate and substrate.

In [9]:
#System
system_inp = Crystal_input(input_name)
system_inp.opt_to_sp()
bsse_sub_inp_name = path+'system.d12'
write_cry_input(bsse_sub_inp_name,system_inp)

shutil.copy(input_name[:-4]+'.gui',path+'system.gui')

#Substrate
indices = sub_ads_indices(system)
bsse_sub_inp = Crystal_input(input_name)
bsse_sub_inp.add_ghost(indices['adsorbate'])
bsse_sub_inp.opt_to_sp()
bsse_sub_inp_name = path+'substrate.d12'
write_cry_input(bsse_sub_inp_name,bsse_sub_inp)

shutil.copy(input_name[:-4]+'.gui',path+'substrate.gui')

#Adsorbate
bsse_ads_inp = Crystal_input(input_name)
bsse_ads_inp.add_ghost(indices['substrate'])
bsse_sub_inp.opt_to_sp()
bsse_ads_inp_name = path+'adsorbate.d12'
write_cry_input(bsse_ads_inp_name,bsse_ads_inp)

shutil.copy(input_name[:-4]+'.gui',path+'adsorbate.gui')

'data/test/adsorbate.gui'

# Run on cx1 

### Check this is fine on cx1
We need to use properties to read the formatted wave function (.f98), write the unformatted one (.f9) and copy that into .f20 that is where the guessp is read from. An issue here could be that the runprop script doesn't copy back the f9. If that is the case we need to modify the script.

In [13]:
density1 = 'data/test/substrate.f98'
density2 = 'data/test/adsorbate.f98'
density3 = 'data/test/system.f98'
cry_combine_density(density1,density2,density3,'data/test/new_density.f98')


file_name = 'system_guessp_p_sum'

prop_block = ['RDFMWF\n','END']
write_cry_properties(path+file_name+'.d3',prop_block)

shutil.copy(path+'new_density.f98',path+'%s.f98'%(file_name))

runprop(path+'%s.d3'%(file_name),path+'%s.f98'%(file_name))

shutil.copy(path+'%s.f9'%(file_name),path+'%s.f20'%(file_name))

#runcry(path+file_name,path+file_name)

'data/test/system_guessp_p_sum.f20'

# Run on cx1 

In [14]:
files = [path+'system.out',path+'system_guessp_p_sum.out']
#,'data/system_guessp_fock0.out']
energy = []
cycles = []
e_ads = []
final_energy = []
for file in files:
    energy.append(Crystal_output(file).final_energy())
    cycles.append(Crystal_output(file).num_cycles())
    e_ads.append(cry_ads_energy(Crystal_output(file).final_energy(),
                                Crystal_output(path+'substrate.out').final_energy(),
                                Crystal_output(path+'adsorbate.out').final_energy()))
    final_energy.append(Crystal_output(file).final_energy())
    

In [15]:
df = pd.DataFrame(list(zip(files,cycles,e_ads,final_energy)),
                 columns=['Name','Cycles','E ads','Final Energy'])
df

,Name,Cycles,E ads,Final Energy
0,data/test/system.out,13,-0.087824,-49131.159455
1,data/test/system_guessp_p_sum.out,7,-0.087825,-49131.159456
